In [1]:
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# from sklearn.naive_bayes import MultinomialNB - хуже работает
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [3]:
%%time
import pickle
with open('page_dict.p', 'rb') as fp:
    page_dict = pickle.load(fp)

CPU times: user 6.14 s, sys: 4.16 s, total: 10.3 s
Wall time: 17.7 s


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
train_data = pd.read_csv('train_groups.csv')
test_data = pd.read_csv('test_groups.csv')

In [6]:
train_documents = []
for i in train_data['doc_id']:
    train_documents.append(page_dict.get(i))

In [7]:
vectorizer = TfidfVectorizer(max_features=500)
vectors = vectorizer.fit_transform(train_documents)

In [8]:
y = np.array(train_data['target'])

In [59]:
clf = LogisticRegression(penalty='l2', random_state=0, C=1.5, max_iter = 100)

In [60]:
cross_val_score(clf, vectors.toarray(), y, cv=5, scoring='f1').mean()

0.4630464967410216

In [61]:
test_documents = []
for i in test_data['doc_id']:
    test_documents.append(page_dict.get(i))

In [62]:
vectors_test = vectorizer.fit_transform(test_documents)

In [63]:
clf = LogisticRegression(penalty='l2', random_state=0)

In [64]:
clf.fit(vectors.toarray(), y)

LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
test_data['target']  =  clf.predict(vectors_test.toarray())

In [66]:
test_data[['pair_id', 'target']].to_csv('try_logreg.csv', index=None)